In [ ]:
import pycantonese, re, shutil

In [ ]:
# List of Korean sentences
sentences = ['你好，我叫張淼。', '下午天氣好熱！', '今天係星期五。', '細仔悟性唔錯。', '呢個係一個好有趣嘅對話。']

# Tokenize each sentence
tokenized_sentences = [' '.join(pycantonese.segment(sentence)) for sentence in sentences]
tokenized_sentences = [re.sub('[。|，|！|？]', '', item) for item in tokenized_sentences]
words = ''.join(tokenized_sentences).split()

# Print the list of tokenized sentences
for i, tokens in enumerate(tokenized_sentences):
    print(f"Sentence {i+1} tokens: {tokens}")

print(words)

In [ ]:
words = ["場", "圖", "到", "嘴", "仔", "光", "⻣", "ㄧ", "㗎", "㗎妹", "㩒", "㩒錢", "㩿", "㪐", "㪐㩿", "䁪", "䁪眼", "䒐䒏", "䟴腳", "䰧", "䱽", "一"]
from transformers import T5ForConditionalGeneration, AutoTokenizer
from lingpy import ipa2tokens

model = T5ForConditionalGeneration.from_pretrained('charsiu/g2p_multilingual_byT5_tiny_16_layers_100')
tokenizer = AutoTokenizer.from_pretrained('google/byt5-small')

chr_words = ['<yue>: '+i for i in words]
out = tokenizer(chr_words,padding=True,add_special_tokens=False,return_tensors='pt')

preds = model.generate(**out,num_beams=1,max_length=50) # We do not find beam search helpful. Greedy decoding is enough. 
phones = tokenizer.batch_decode(preds.tolist(),skip_special_tokens=True)


phones = [ipa2tokens(phone) for phone in phones]
phones = [' '.join(phone) for phone in phones]
phones = [re.sub(':', 'ː', phone) for phone in phones]

    
# Define the regular expression pattern to capture whitespace between 't' and 's' following IPA tone letters
pattern_ts = re.compile(r'(^|[˥˦˧˨˩]\s)t s')
pattern_ng_syll = re.compile(r'(^|[˥˦˧˨˩]+\s)ŋ(\s[˥˦˧˨˩]+)')

# Replace the whitespace between 't' and 's' with empty string
phones = [re.sub(pattern_ts, r'\1t͡s', phone) for phone in phones]
phones = [re.sub(pattern_ng_syll, r'\1ŋ̩\2', phone) for phone in phones]

pattern_tone_pos = re.compile(r'\s([mnŋptk])\s([˥˦˧˨˩]+)')
#pattern_tone_strip = re.compile(r'\s[˥˦˧˨˩]+(^|\s)')

phones_with_tone = [re.sub(pattern_tone_pos, r' \2 \1', phone) for phone in phones]
phones_with_tone = [re.sub(r'(.)\s([˥˦˧˨˩]+)', r'\1\2', phone) for phone in phones_with_tone]
phones_no_tone = [re.sub(r'[˥˦˧˨˩]+', '', phone) for phone in phones]
phones_no_tone = [re.sub(r'\s+', ' ', phone) for phone in phones_no_tone]

for word, phone in zip(words, phones_with_tone):
    print(word + '\t' + phone)
print("\n")
for word, phone in zip(words, phones_no_tone):
    print(word + '\t' + phone)


In [ ]:
# Read and combine the content of both files
combined_content = ""
with open('/Users/miaozhang/Research/CorpusPhon/CorpusData/CommonVoice/zh-HK_v17/zh-HK_chr_lexicon17.txt', 'r') as file1:
    combined_content += file1.read()

with open('/Users/miaozhang/Research/CorpusPhon/CorpusData/CommonVoice/yue_v17/yue_chr_lexicon17.txt', 'r') as file2:
    combined_content += file2.read()

# Get unique values and sort them
unique_sorted_content = '\n'.join(sorted(set(combined_content.splitlines())))

# Write the unique and sorted content to a new file
with open('/Users/miaozhang/Research/CorpusPhon/CorpusData/CommonVoice/zh-HK_v17/cantonese_chr_lexicon17.txt', 'w') as output_file:
    output_file.write(unique_sorted_content)
shutil.copy('/Users/miaozhang/Research/CorpusPhon/CorpusData/CommonVoice/zh-HK_v17/cantonese_chr_lexicon17.txt', '/Users/miaozhang/Research/CorpusPhon/CorpusData/CommonVoice/yue_v17/cantonese_chr_lexicon17.txt')